In [12]:
!pip install mysql-connector-python
!pip uninstall -y langchain langchain-community langchain-core
!pip install langchain==0.1.20
!pip install langchain-community==0.0.36
!pip install langchain-core==0.1.52
!pip install langchain-text-splitters
!pip install transformers sentence-transformers faiss-cpu pypdf

Found existing installation: langchain 0.1.20
Uninstalling langchain-0.1.20:
  Successfully uninstalled langchain-0.1.20
Found existing installation: langchain-community 0.0.38
Uninstalling langchain-community-0.0.38:
  Successfully uninstalled langchain-community-0.0.38
Found existing installation: langchain-core 0.1.52
Uninstalling langchain-core-0.1.52:
  Successfully uninstalled langchain-core-0.1.52
  Using cached langchain-0.1.20-py3-none-any.whl.metadata (13 kB)
  Using cached langchain_community-0.0.38-py3-none-any.whl.metadata (8.7 kB)
  Using cached langchain_core-0.1.53-py3-none-any.whl.metadata (5.9 kB)
Using cached langchain-0.1.20-py3-none-any.whl (1.0 MB)
Using cached langchain_community-0.0.38-py3-none-any.whl (2.0 MB)
Using cached langchain_core-0.1.53-py3-none-any.whl (303 kB)


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain-classic 1.0.0 requires langchain-core<2.0.0,>=1.0.0, but you have langchain-core 0.1.53 which is incompatible.
langchain-classic 1.0.0 requires langchain-text-splitters<2.0.0,>=1.0.0, but you have langchain-text-splitters 0.0.2 which is incompatible.
langgraph-checkpoint 3.0.1 requires langchain-core>=0.2.38, but you have langchain-core 0.1.53 which is incompatible.
langgraph-prebuilt 1.0.5 requires langchain-core>=1.0.0, but you have langchain-core 0.1.53 which is incompatible.


  Using cached langchain_community-0.0.36-py3-none-any.whl.metadata (8.7 kB)
Using cached langchain_community-0.0.36-py3-none-any.whl (2.0 MB)
  Attempting uninstall: langchain-community
    Found existing installation: langchain-community 0.0.38
    Uninstalling langchain-community-0.0.38:
      Successfully uninstalled langchain-community-0.0.38


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain 0.1.20 requires langchain-community<0.1,>=0.0.38, but you have langchain-community 0.0.36 which is incompatible.


  Using cached langchain_core-0.1.52-py3-none-any.whl.metadata (5.9 kB)
Using cached langchain_core-0.1.52-py3-none-any.whl (302 kB)
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.1.53
    Uninstalling langchain-core-0.1.53:
      Successfully uninstalled langchain-core-0.1.53


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain 0.1.20 requires langchain-community<0.1,>=0.0.38, but you have langchain-community 0.0.36 which is incompatible.
langchain-classic 1.0.0 requires langchain-core<2.0.0,>=1.0.0, but you have langchain-core 0.1.52 which is incompatible.
langchain-classic 1.0.0 requires langchain-text-splitters<2.0.0,>=1.0.0, but you have langchain-text-splitters 0.0.2 which is incompatible.
langgraph-checkpoint 3.0.1 requires langchain-core>=0.2.38, but you have langchain-core 0.1.52 which is incompatible.
langgraph-prebuilt 1.0.5 requires langchain-core>=1.0.0, but you have langchain-core 0.1.52 which is incompatible.


In [13]:
#Load Data

import pandas as pd
import mysql.connector

conn = mysql.connector.connect(
    host='127.0.0.1',
    port=3306,
    user='root',
    password='S@i300498',
    database='shop_inventory'
)

query = "SELECT * FROM products;"  
df = pd.read_sql(query, conn)

conn.close()
print(df.head())

   product_id product_name product_brand product_size  product_mrp
0           1         Milk          Amul           1L         60.0
1           2        Bread     Britannia         400g         40.0
2           3         Eggs     LocalFarm        12pcs         90.0
3           4         Rice        Daawat          5kg        450.0
4           5        Apple    FreshFarms          1kg        180.0


C:\Users\admine\AppData\Local\Temp\ipykernel_7004\3340500437.py:15: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


In [14]:
# Imports
import re
import requests
import logging

from langchain_core.documents import Document
from langchain_text_splitters import RecursiveCharacterTextSplitter

from langchain_community.llms import HuggingFacePipeline
from langchain.chains import RetrievalQA

from transformers import pipeline
from transformers import logging as hf_logging

from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS

In [15]:
hf_logging.set_verbosity_error()

In [17]:

logging.basicConfig(level=logging.INFO)

legal_stopwords = {
    "the","herein","hereinafter","therein","thereof","thereby","therewith",
    "whereas","wherein","whereof","thereafter","thereunder","hereunder",
    "hereafter","hereto","thereto","hereby","thereby","aforementioned",
    "notwithstanding","pursuant","agreement","party","parties","contract",
    "terms","conditions","plaintiff","defendant","court","judge","jury",
    "proceedings","appeal","case","order","decision","shall","may","must",
    "will"
}

def clean_legaltext(text):
    if not text or not isinstance(text, str):
        return ' '
    text = text.lower()
    text = re.sub(r'\s+', ' ', text)
    text = re.sub(r'[^a-zA-Z0-9\s]', ' ', text)
    # Only keep words that are not stopwords AND longer than 2 characters
    words = [word for word in text.split() if word not in legal_stopwords and len(word) > 2]
    return ' '.join(words)

def preprocess_documents(documents):
    """
    Takes a list of Document objects and returns a list of cleaned Document objects.
    """
    preprocessed_docs = []
    for doc in documents:
        cleaned_text = clean_legaltext(doc.page_content)
        new_doc = Document(page_content=cleaned_text, metadata=doc.metadata)
        preprocessed_docs.append(new_doc)
    return preprocessed_docs

documents = [Document(page_content=row, metadata={"source": "products"}) for row in df['product_name']]

processed_documents = preprocess_documents(documents)


In [18]:
# ==== Chunking ====

text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=200)

def chunk_documents(documents):
    chunks = []
    for doc in documents:
        text_chunks = text_splitter.split_text(doc.page_content)
        for i, chunk in enumerate(text_chunks):
            new_metadata = doc.metadata.copy()
            new_metadata["chunk_id"] = i
            chunks.append(Document(page_content=chunk, metadata=new_metadata))
    return chunks

documents_chunked = chunk_documents(processed_documents)

In [19]:
# ==== Embeddings ====

embedding_function = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2"
)

INFO:sentence_transformers.SentenceTransformer:Use pytorch device_name: cpu
INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: sentence-transformers/all-MiniLM-L6-v2


In [20]:
# ==== Vector Store ====

vectorstore = FAISS.from_documents(documents_chunked, embedding_function)

INFO:faiss.loader:Loading faiss with AVX2 support.
INFO:faiss.loader:Successfully loaded faiss with AVX2 support.


In [21]:
# ==== LLM ====

llm_pipeline = pipeline(
    "text-generation",
    model="tiiuae/falcon-7b-instruct",
    device='cpu',
    max_new_tokens=150,
    temperature=0.5
)

llm = HuggingFacePipeline(pipeline=llm_pipeline)

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model-00002-of-00002.safetensors:  83%|########3 | 3.73G/4.48G [00:00<?, ?B/s]

model-00001-of-00002.safetensors:  24%|##4       | 2.41G/9.95G [00:00<?, ?B/s]

C:\Users\admine\anaconda3\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\admine\.cache\huggingface\hub\models--tiiuae--falcon-7b-instruct. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/117 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/281 [00:00<?, ?B/s]

In [24]:
# ==== Retriever & QA Chain ====

retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 3})

rag_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    chain_type="map_reduce"
)

In [ ]:
# ==== User Query ====

query = input("How may i assist you: ")

result = rag_chain.invoke({"query": query})
print(result["result"])


How may i assist you:  where can i find amul milk?
